# Import packages

In [1]:
import argparse
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display
from multiprocessing import cpu_count,Pool 
from joblib import Parallel, delayed
import psutil
from sklearn.cluster import KMeans, SpectralClustering, DBSCAN 
from sklearn.neighbors import NearestNeighbors
from itertools import cycle, islice
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import gmaps
#import hdbscan
import seaborn as sns


# define helper functions

In [2]:
def print_memory_usage():
    print ("memory log:")
    process = psutil.Process(os.getpid())
    print("%5.2f GB (RSS)" % (process.memory_info().rss / 2**30))
    print("%5.2f GB (VMS)" % (process.memory_info().vms / 2**30))
    print("%5.2f GB (Used)" % (psutil.virtual_memory().used / 2**30))
    print("%5.2f GB (Available)" % (psutil.virtual_memory().available / 2**30))
    print("%5.2f GB (Total)" % (psutil.virtual_memory().total / 2**30))

# set parameters

In [3]:
cores = cpu_count() #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want
print ("number of cores "+str(partitions))

number of cores 28


# load data

In [4]:
import codecs
print ("load data set")
data_set = pd.read_hdf('../../data_set.h5',key='DS_new')
print ("Data set size: ", data_set.shape[0])
#display(data_set)
print ("*"*80)
# filter data

load data set
Data set size:  15325090
********************************************************************************


# get sample 

In [5]:
traffic_set = data_set[data_set.Type == 'T']
print (traffic_set.shape[0])
traffic_set = traffic_set.sample(n=2000000,replace=True)
print ('after sampling ',traffic_set.shape[0])

13147811
after sampling  2000000


In [6]:
traffic_set.to_hdf('../../data_set.h5',key='DS_new_sampled')

/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['EventId', 'Type', 'RefinedType_', 'AirportCode', 'Street', 'Side', 'City', 'County', 'State', 'RefinedType']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


## get traffic data and select whole or subset of it.

In [7]:
coords = traffic_set.as_matrix(columns=['LocationLat', 'LocationLng'])
print ('Loaded {} records'.format(len(coords)))
data = np.radians(coords)

Loaded 2000000 records


kms_per_radian = 3959
epsilon = 4.09/ kms_per_radian #default: 4.09 km as neighborhood threshold!
clusterer = DBSCAN(eps=epsilon, min_samples=10, algorithm='auto', metric='haversine',n_jobs=cores).fit(data)
cluster_labels = clusterer.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

## set parameters and make clusterer

In [8]:
kms_per_radian = 3959
epsilon = 4.09 / kms_per_radian #default: 1.5 km as neighborhood threshold!
clusterer = DBSCAN(eps=epsilon, min_samples=463, algorithm='auto', metric='haversine',n_jobs=cores).fit(data)

# store the result of clustering

In [9]:
np.save("../../clustering_cordinates",data)
loaded_data = np.load("../../clustering_cordinates"+".npy")
np.save("../../cluster_labels",clusterer.labels_)
loaded_labels = np.load("../../cluster_labels"+".npy")